In [2]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [18]:
day = 111209
detectors = np.array(['n3', 'n6', 'n7', 'n8', 'nb'])
#160624
time_frame = np.array([7.75775, 7.75781])
#140611
#time_frame = np.array([4.7185, 4.7285])
#101225
#time_frame = np.array([9.0481, 9.078])
data_type = 'ctime'
echans = np.array([1, 2, 3, 4, 5, 6, 7])

In [19]:
all_det_counts = []
for echan in echans:
    det_counts = []
    for detector in detectors:
        data = calc.curve_fit_plots(day, detector, echan, data_type)
        residuals = data[0]
        plot_time_bin = data[8]
        plot_time_sat = data[9]
        
        residuals_frame = residuals[np.where(np.logical_and(plot_time_bin > time_frame[0], plot_time_bin < time_frame[1]))]
        det_counts.append(np.sum(residuals_frame))
    
    all_det_counts.append(det_counts)

all_det_counts = np.array(all_det_counts)
mid_point_index = ([np.where(np.logical_and(plot_time_bin > time_frame[0], plot_time_bin < time_frame[1]))][0][0][0] + [np.where(np.logical_and(plot_time_bin > time_frame[0], plot_time_bin < time_frame[1]))][0][0][-1])/2

In [20]:
det_coords = []
det_rads = []
for detector in detectors:
    det_or = calc.det_or_bin(detector, day, 0, data_type)
    det_coor = det_or[0]
    det_rad = det_or[1]
    sat_pos = det_or[2]
    
    det_coords.append(det_coor[:,mid_point_index])
    det_rads.append(det_rad[mid_point_index])

det_coords = np.array(det_coords)
det_rads = np.array(det_rads)

In [21]:
def angle(x1, x2, x3, coords):
    x_vec = np.array([x1, x2, x3])
    #normalize vektor
    x_vec = x_vec/(np.sqrt(x1**2 + x2**2 + x3**2))
    
    scalar_product = x_vec[0]*coords[0] + x_vec[1]*coords[1] + x_vec[2]*coords[2]
    angle = np.arccos(scalar_product)*360./(2.*math.pi)
    return angle


def eff_counts(angle, echan, factor):
    fitsname = 'peak_eff_area_angle_calib_GBM_all_DRM.fits'
    user = getpass.getuser()
    fits_path = '/home/' + user + '/Work/calibration/'
    fitsfilepath = os.path.join(fits_path, fitsname)
    fitsfile = fits.open(fitsfilepath, mode='update')
    data = fitsfile[1].data
    fitsfile.close()
    x5 = data.field(0)
    y0 = data.field(1)#for NaI (4-12 keV) gewichteter Mittelwert = 10.76 keV
    y1 = data.field(2)#for NaI (12-27 keV) gewichteter Mittelwert = 20.42 keV
    y2 = data.field(3)#for NaI (27-50 keV) gewichteter Mittelwert = 38.80 keV
    y3 = data.field(4)#for NaI (50-102 keV) gewichteter Mittelwert = 76.37 keV
    y4 = data.field(5)#for NaI (102-295 keV) gewichteter Mittelwert = 190.19 keV
    y5 = data.field(6)#for NaI (295-540 keV) gewichteter Mittelwert = 410.91 keV
    y6 = data.field(7)#for NaI (540-985 keV) gewichteter Mittelwert = 751.94 keV
    y7 = data.field(8)#for NaI (985-2000 keV) gewichteter Mittelwert = 1466.43 keV
    #y4 = data.field(4)#for BGO (898 keV)
    #y5 = data.field(5)#for BGO (1836 keV)
        
    y_all = np.array([y0, y1, y2, y3, y4, y5, y6, y7])
    ang_eff = []
        
    y = y_all[echan]
                
    #calculate the angle factors
    tck = interpolate.splrep(x5, y)
    counts_fac = interpolate.splev(angle, tck, der=0)
    counts_fac = counts_fac*factor
    return counts_fac


def echan_fit_function(x, det_coords, echan, x1, x2, x3, factor):
    det_counts_fit = []
    for coords in det_coords:
        ang = angle(x1, x2, x3, coords)
        counts_fac = eff_counts(ang, echan, factor)
        det_counts_fit.append(counts_fac)
    
    det_counts_fit = np.array(det_counts_fit)
    return det_counts_fit

def fit_function(x, x_angles, det_coords, echans, x1, x2, x3, factor):
    all_det_counts_fit = []
    for echan in echans:
        det_counts_fit = echan_fit_function(x, det_coords, echan, x1, x2, x3, factor)
        all_det_counts_fit.append(det_counts_fit)
    
    all_det_counts_fit = np.array(all_det_counts_fit)
    return all_det_counts_fit

In [22]:
results_rad = []
results_coor = []
results_factor = []
for i in range(0, len(echans)):
    echan_index = i
    echan = echans[echan_index]

    x_axis = np.arange(len(detectors))
    y_axis = all_det_counts[echan_index]

    x0 = np.array([1., 1., 1., 1000.])

    fit_results = optimization.curve_fit(lambda x, x1, x2, x3, factor: echan_fit_function(x, det_coords, echan, x1, x2, x3, factor), x_axis, y_axis, x0, maxfev = 10000)
    best_coords = fit_results[0][:3]
    factor = fit_results[0][3]
    #print LA.norm(best_coords, axis=0)
    best_coords = best_coords/LA.norm(best_coords, axis=0)
    best_ra = np.arctan2(-best_coords[1], -best_coords[0])*360./(2.*math.pi)+180.
    best_dec = np.arctan(best_coords[2]/(best_coords[0]**2 + best_coords[1]**2)**0.5)*360./(2.*math.pi)
    best_rad = np.array([best_ra, best_dec])
    
    results_rad.append(best_rad)
    results_coor.append(best_coords)
    results_factor.append(factor)

results_rad = np.array(results_rad)
results_coor = np.array(results_coor)
results_factor = np.array(results_factor)

In [23]:
print results_rad
print results_coor
print results_factor

[[  33.4373388   -20.59303003]
 [  15.15303424  -24.07248667]
 [ 207.03502076   -0.30196285]
 [  63.27045704   14.25977536]
 [  12.00481299  -32.02679524]
 [ 295.25071166  -20.28708979]
 [ 149.10533874   81.00902121]]
[[ 0.78116705  0.51581549 -0.35172778]
 [ 0.88128509  0.23866431 -0.40789207]
 [-0.8907165  -0.45452871 -0.00527022]
 [ 0.43592138  0.86562098  0.24631865]
 [ 0.8292589   0.17633723 -0.53031581]
 [ 0.40011797 -0.84834419 -0.34672431]
 [-0.13410496  0.08024319  0.98771296]]
[ 0.64000755  0.63028533  1.36228908  0.92910883  0.38606714  1.83307615
  3.54362469]


In [93]:
print results_rad
print results_coor
print results_factor

[[ 233.17807541  -44.51957995]
 [ 302.94449035   -6.26165151]
 [ 341.8537932     1.71264766]
 [ 338.8191877    12.88451741]
 [ 285.40108561   18.45226637]
 [ 327.97658155   -9.81906695]
 [ 244.00137136  -18.39737431]]
[[-0.42732878 -0.5707667  -0.70115297]
 [ 0.54058188 -0.83419133 -0.10906902]
 [ 0.94984038 -0.31130376  0.02988689]
 [ 0.90896728 -0.35221502  0.22298671]
 [ 0.25192059 -0.91452425  0.31651449]
 [ 0.83541172 -0.52249808 -0.17053741]
 [-0.41594579 -0.85286707 -0.31560555]]
[ 1172.73628304   604.75858246   768.49731081  1023.55371131   432.01720063
   389.59453946   363.27310692]


In [9]:
all_det_counts_fit = []
for i in range(0, len(echans)):
    det_counts_fit = echan_fit_function(0, det_coords, echan, results_coor[i][0], results_coor[i][1], results_coor[i][2], results_factor[i])
    all_det_counts_fit.append(det_counts_fit)

all_det_counts_fit = np.array(all_det_counts_fit)

In [ ]:
print all_det_counts_fit

In [ ]:
print all_det_counts

In [10]:
print (all_det_counts - all_det_counts_fit)/all_det_counts

[[ -1.08099722e+01   1.42476407e+01  -2.29634401e+00  -8.01836473e+00
   -2.26615448e+00  -3.52085117e+00   1.57524982e-01   1.94887819e-01
   -5.90682478e-01  -4.45413196e-02  -4.75029771e+00   3.61672006e+00]
 [ -1.31235589e+00  -3.48021007e-01  -4.29917453e+02  -5.14556789e-02
   -2.35286394e-01   1.31736605e-01  -1.59264282e+00  -3.10350477e-01
    1.00063333e-01   2.91677810e-01   3.12938440e-01  -1.00548403e+01]
 [ -9.00953220e-01  -2.39063383e-01  -1.53094304e+00  -1.06926778e-01
   -4.71863382e-01   2.61936653e-01  -3.48692781e+00  -1.07823404e+00
   -3.01882857e-02  -1.50769083e-01  -2.35903016e-01  -6.56125837e-01]
 [ -2.73476486e-01  -1.48147081e-01  -7.89038074e-01  -1.37651739e+00
    3.78843963e-01  -2.58529869e-01  -1.34430356e+00  -1.51795938e-01
   -9.42032653e-01  -7.38806656e-02  -6.67727624e-01   5.39940264e-01]
 [  4.27329127e-01  -5.58145371e-02  -8.72085127e-01   2.60722823e-01
    2.49721053e-01  -2.54378239e-01  -7.28317404e-02   4.60593757e-02
   -4.87136036e-